System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.10.2
Commit bd47eca2c8a (2024-03-01 10:14 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2024spring/slides/02-langs`


Status `~/Documents/github.com/ucla-biostat-257/2024spring/slides/02-langs/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [31c24e10] Distributions v0.25.107
  [bdcacae8] LoopVectorization v0.12.166
  [438e738f] PyCall v1.96.4
  [6f49c342] RCall v0.14.1
  [8f399da3] Libdl
  [9a3f8284] Random
  [10745b16] Statistics v1.10.0


In this lecture, we review some popular computer languages commonly used in computational statistics. We want to understand why some languages are faster than others and what are the remedies for the slow languages.

## Types of computer languages

* **Compiled languages** (low-level languages): C/C++, Fortran, ... 
  - Directly compiled to machine code that is executed by CPU 
  - Pros: fast, memory efficient
  - Cons: longer development time, hard to debug

* **Interpreted language** (high-level languages): R, Matlab, Python, SAS IML, JavaScript, ... 
  - Interpreted by interpreter
  - Pros: fast prototyping
  - Cons: excruciatingly slow for loops

* Mixed (dynamic) languages: Matlab (JIT), R (`compiler` package), Julia, Cython, JAVA, ...
  - Pros and cons: between the compiled and interpreted languages

* Script languages: Linux shell scripts, Perl, ...
  - Extremely useful for some data preprocessing and manipulation

* Database languages: SQL, Hive (Hadoop).  
  - Data analysis *never* happens if we do not know how to retrieve data from databases  
  
  [203B (Introduction to Data Science)](https://ucla-biostat-203b.github.io/2024winter/schedule/schedule-lec1.html) covers some basics on scripting and database.

## How high-level languages work?

Comics: [The Life of a Bytecode Language](https://betterprogramming.pub/the-life-of-a-bytecode-language-fca666928e7b)

- Typical execution of a high-level language such as R, Python, and Matlab.

<img src="./r-bytecode.png" align="center" width="400"/>

- To improve efficiency of interpreted languages such as R or Matlab, conventional wisdom is to avoid loops as much as possible. Aka, **vectorize code**
> The only loop you are allowed to have is that for an iterative algorithm.

- When looping is unavoidable, need to code in C, C++, or Fortran. This creates the notorious **two language problem**  
Success stories: the popular `glmnet` package in R is coded in Fortran; `tidyverse` and `data.table` packages use a lot RCpp/C++.

<img src="./two_language_problem.jpg" align="center" width="600"/>

- High-level languages have made many efforts to bring themselves closer to the performance of low-level languages such as C, C++, or Fortran, with a variety levels of success.   
    - Matlab has employed JIT (just-in-time compilation) technology since 2002.  
    - Since R 3.4.0 (Apr 2017), the JIT bytecode compiler is enabled by default at its level 3.   
    - Cython is a compiler system based on Python.  

- Modern languages such as Julia tries to solve the two language problem. That is to achieve efficiency without vectorizing code.

- Julia execution.

<img src="./julia_compile.png" align="center" width="400"/>

<img src="./julia_introspect.png" align="center" width="400"/>

## Gibbs sampler example by Doug Bates

- Doug Bates (member of R-Core, author of popular R packages `Matrix`, `lme4`, `RcppEigen`, etc)
    
    > As some of you may know, I have had a (rather late) mid-life crisis and run off with another language called Julia.   
    >
    > -- <cite>Doug Bates (on the `knitr` Google Group)</cite>

- An example from Dr. Doug Bates's slides [Julia for R Programmers](http://www.stat.wisc.edu/~bates/JuliaForRProgrammers.pdf).

- The task is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 \exp(- x y^2 - y^2 + 2y - 4x), \quad x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

* R solution. The `RCall.jl` package allows us to execute R code without leaving the `Julia` environment. We first define an R function `Rgibbs()`.

In [3]:
using RCall

# show R information
R"""
sessionInfo()
"""

RObject{VecSxp}
R version 4.3.2 (2023-10-31)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS Sonoma 14.4

Matrix products: default
BLAS:   /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
[1] compiler_4.3.2


In [4]:
# define a function for Gibbs sampling

R"""
library(Matrix)

Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
"""

RObject{ClosSxp}
function (N, thin) 
{
    mat <- matrix(0, nrow = N, ncol = 2)
    x <- y <- 0
    for (i in 1:N) {
        for (j in 1:thin) {
            x <- rgamma(1, 3, y * y + 4)
            y <- rnorm(1, 1/(x + 1), 1/sqrt(2 * (x + 1)))
        }
        mat[i, ] <- c(x, y)
    }
    mat
}


To generate a sample of size 10,000 with a thinning of 500. How long does it take?

In [5]:
R"""
system.time(Rgibbs(10000, 500))
"""

RObject{RealSxp}
   user  system elapsed 
 10.941   0.727  11.934 


* This is a Julia function for the same Gibbs sampler:

In [6]:
using Distributions

function jgibbs(N, thin)
    mat = zeros(N, 2)
    x = y = 0.0
    for i in 1:N
        for j in 1:thin
            x = rand(Gamma(3, 1 / (y * y + 4)))
            y = rand(Normal(1 / (x + 1), 1 / sqrt(2(x + 1))))
        end
        mat[i, 1] = x
        mat[i, 2] = y
    end
    mat
end

jgibbs (generic function with 1 method)

Generate the same number of samples. How long does it take?

In [7]:
jgibbs(100, 5); # warm-up
@elapsed jgibbs(10000, 500)

0.112395084

We see 40-80 fold speed up of `Julia` over `R` on this example, **with similar coding effort**!

## Comparing C, C++, R, Python, and Julia

To better understand how these languages work, we consider a simple task: summing a vector.

Let's first generate data: 1 million double precision numbers from uniform [0, 1].

In [8]:
using Random

Random.seed!(257) # seed
x = rand(1_000_000) # 1 million random numbers in [0, 1)
sum(x)

500156.3977288406

In this class, we extensively use package `BenchmarkTools.jl` for robust benchmarking. It's the analog of the `microbenchmark` or `bench` package in R.

In [9]:
using BenchmarkTools

### C

We would use the low-level C code as the baseline for copmarison. In Julia, we can easily run compiled C code using the `ccall` function.

In [10]:
using Libdl

C_code = """
#include <stddef.h>

double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):
open(`gcc -std=c99 -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X :: Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

clang: warning: argument unused during compilation: '-msse3' [-Wunused-command-line-argument]


c_sum (generic function with 1 method)

In [11]:
# make sure it gives same answer
c_sum(x)

500156.39772885485

In [12]:
# dollar sign to interpolate array x into local scope for benchmarking
bm = @benchmark c_sum($x)

BenchmarkTools.Trial: 5999 samples with 1 evaluation.
 Range (min … max):  811.625 μs … 993.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     823.958 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   829.585 μs ±  20.845 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▆▃▅▅▅▅▄▄▄▄▄▄▃▄▃▃▃▂▂▂▂▁▁▁▁▁▁ ▁                               ▂
  ███████████████████████████████▇████▇▆▆▇▆▆▆▆▅▆▆▅▅▆▅▆▆▅▆▅▅▇▄▅▅ █
  812 μs        Histogram: log(frequency) by time        911 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
# a dictionary to store median runtime (in milliseconds)
benchmark_result = Dict() 

# store median runtime (in milliseconds)
benchmark_result["C"] = median(bm.times) / 1e6

0.823958

### R, buildin sum

Next we compare to the build in `sum` function in R, which is implemented using C.

In [14]:
R"""
library(bench)
library(tidyverse)

# interpolate x into R workspace
y <- $x
rbm_builtin <- bench::mark(sum(y)) %>%
  print(width = Inf)
""";

┌ Warning: RCall.jl: -- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
│ v dplyr     1.1.3     v readr     2.1.4
│ v forcats   1.0.0     v stringr   1.5.0
│ v ggplot2   3.4.4     v tibble    3.2.1
│ v lubridate 1.9.2     v tidyr     1.3.0
│ v purrr     1.0.2     
│ -- Conflicts ------------------------------------------ tidyverse_conflicts() --
│ x tidyr::expand() masks Matrix::expand()
│ x dplyr::filter() masks stats::filter()
│ x dplyr::lag()    masks stats::lag()
│ x tidyr::pack()   masks Matrix::pack()
│ x tidyr::unpack() masks Matrix::unpack()
│ i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
└ @ RCall /Users/huazhou/.julia/packages/RCall/dDAVd/src/io.jl:172


# A tibble: 1 x 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 sum(y)       1.33ms   1.37ms      720.        0B        0   360     0
  total_time result    memory             time             gc                
    <bch:tm> <list>    <list>             <list>           <list>            
1      500ms <dbl [1]> <Rprofmem [0 x 3]> <bench_tm [360]> <tibble [360 x 3]>


In [15]:
# store median runtime (in milliseconds)
@rget rbm_builtin # dataframe
benchmark_result["R builtin"] = median(rbm_builtin[!, :median]) * 1000

1.3731925282627344

### R, handwritten loop

Handwritten loop in R is much slower.

In [16]:
R"""
sum_r <- function(x) {
  s <- 0
  for (xi in x) {
    s <- s + xi
  }
  s
}
library(bench)
y <- $x
rbm_loop <- bench::mark(sum_r(y)) %>%
  print(width = Inf)
""";

# A tibble: 1 x 13
  expression      min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr> <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 sum_r(y)     7.08ms    7.6ms      132.    13.1KB        0    66     0
  total_time result    memory              time            gc               
    <bch:tm> <list>    <list>              <list>          <list>           
1      501ms <dbl [1]> <Rprofmem [42 x 3]> <bench_tm [66]> <tibble [66 x 3]>


In [17]:
# store median runtime (in milliseconds)
@rget rbm_loop # dataframe
benchmark_result["R loop"] = median(rbm_loop[!, :median]) * 1000

7.598653028253466

### R, Rcpp

Rcpp package provides an easy way to incorporate C++ code in R.

In [18]:
R"""
library(Rcpp)

cppFunction('double rcpp_sum(NumericVector x) {
  int n = x.size();
  double total = 0;
  for(int i = 0; i < n; ++i) {
    total += x[i];
  }
  return total;
}')

rcpp_sum
"""

RObject{ClosSxp}
function (x) 
.Call(<pointer: 0x3491c08c0>, x)


In [19]:
R"""
rbm_rcpp <- bench::mark(rcpp_sum(y)) %>%
  print(width = Inf)
""";

# A tibble: 1 x 13
  expression       min   median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc
  <bch:expr>  <bch:tm> <bch:tm>     <dbl> <bch:byt>    <dbl> <int> <dbl>
1 rcpp_sum(y)    800us    852us     1154.    2.49KB        0   577     0
  total_time result    memory             time             gc                
    <bch:tm> <list>    <list>             <list>           <list>            
1      500ms <dbl [1]> <Rprofmem [1 x 3]> <bench_tm [577]> <tibble [577 x 3]>


In [20]:
# store median runtime (in milliseconds)
@rget rbm_rcpp # dataframe
benchmark_result["R Rcpp"] = median(rbm_rcpp[!, :median]) * 1000

0.8523900178261101

### Python, builtin sum

Built in function `sum` in Python.

In [21]:
using PyCall

PyCall.pyversion

v"3.10.12"

In [22]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
bm = @benchmark $pysum($x)

BenchmarkTools.Trial: 129 samples with 1 evaluation.
 Range (min … max):  37.617 ms …  39.755 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     38.883 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.813 ms ± 447.683 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                ▂ ▂   ▂     ▄   ▆    ▂▆▂██ ▄█    ▄▂      ▂      
  ▆▁▁▄▁▁▁▁▄▄▆▁▄▁█▄█▄▁▄█▄▄▄▆▄█▆█▄███▆▆█████▄███▆████▆▁▄▄▄▁█▁▄▁▄ ▄
  37.6 ms         Histogram: frequency by time         39.7 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [23]:
# store median runtime (in miliseconds)
benchmark_result["Python builtin"] = median(bm.times) / 1e6

38.883459

### Python, handwritten loop

In [24]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

bm = @benchmark $sum_py($x)

BenchmarkTools.Trial: 105 samples with 1 evaluation.
 Range (min … max):  45.974 ms …  48.898 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     47.732 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   47.631 ms ± 545.860 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                    ▁▄  ▁ ▂ █  ▁                
  ▃▁▁▁▁▁▁▃▁▁▃▃▁▁▅▃▅▃▅▅▅▅▅▅▅▃▁▆▅▃▁▆▃███▆▆█▆███▆▅█▅▃▁▃▅▁▁▃▃▁▁▁▁▃ ▃
  46 ms           Histogram: frequency by time         48.8 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [25]:
# store median runtime (in miliseconds)
benchmark_result["Python loop"] = median(bm.times) / 1e6

47.732083

### Python, numpy

Numpy is the high-performance scientific computing library for Python.

In [26]:
# bring in sum function from Numpy 
numpy_sum = pyimport("numpy")."sum"

PyObject <function sum at 0x34a36ea30>

In [27]:
bm = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  160.708 μs … 279.334 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     173.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   176.551 μs ±   5.542 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                      ▇█▅▃▁ ▅▄▄▃▂▁▁▂▃▃▃▃▂▁▁▁▂▁▁▁                ▂
  ▄▄▁▄▃▃▁▁▁▁▃▁▁▃▁▁▁▁▁▄███████████████████████████████████▇▇▆▆▆▆ █
  161 μs        Histogram: log(frequency) by time        197 μs <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [28]:
# store median runtime (in miliseconds)
benchmark_result["Python numpy"] = median(bm.times) / 1e6

0.17375

Numpy performance is on a par with Julia build-in sum function. Both are about 3 times faster than C, possibly because of usage of [SIMD](https://en.wikipedia.org/wiki/SIMD).

### Julia, builtin sum

`@time`, `@elapsed`, `@allocated` macros in Julia report run times and memory allocation.

In [29]:
@time sum(x) # no compilation time after first run

  0.000184 seconds (1 allocation: 16 bytes)


500156.3977288406

For more robust benchmarking, we use BenchmarkTools.jl package.

In [30]:
bm = @benchmark sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  159.833 μs … 228.250 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     160.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   164.049 μs ±   6.280 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁█▅     ▄▃▃▂▁   ▁▂▂▁▁▂▁     ▁▁▁                               ▁
  ███▇▆▇▆▅██████▇████████████████▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▇▆▅▅▆▅▆▆▅▃▅▄▅▃▃ █
  160 μs        Histogram: log(frequency) by time        188 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [31]:
benchmark_result["Julia builtin"] = median(bm.times) / 1e6

0.16075

### Julia, handwritten loop

Let's also write a loop and benchmark.

In [32]:
function jl_sum(A)
    s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

bm = @benchmark jl_sum($x)

BenchmarkTools.Trial: 5969 samples with 1 evaluation.
 Range (min … max):  811.625 μs …  1.042 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     829.500 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   836.934 μs ± 26.608 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                             
  █▄▅▅▇▆▇▅▇▅▆▅▅▄▄▄▃▃▃▂▃▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  812 μs          Histogram: frequency by time          931 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
benchmark_result["Julia loop"] = median(bm.times) / 1e6

0.8295

**Exercise**: annotate the loop by [`@simd`](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations-1) or `LoopVectorization.@turbo` and benchmark again. 

In [34]:
using LoopVectorization

function jl_sum_turbo(A)
    s = zero(eltype(A))
    @turbo for i in eachindex(A)
      s += A[i]
    end
    s
end

bm = @benchmark jl_sum_turbo($x)
bm

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  81.625 μs … 167.416 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     89.750 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   90.455 μs ±   4.697 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▃▄▅▂            ▁▅▇█▃▁    ▂▄▂▂▂▁▁   ▁▂▁▁▁ ▁                 ▂
  ▅████▆▆▅▅▅▇███▇▇▆████████▇█████████▇▇█████████▇▇██▇▇██▇▆▆▆▆▆ █
  81.6 μs       Histogram: log(frequency) by time       106 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [35]:
benchmark_result["Julia turbo"] = median(bm.times) / 1e6

0.08975

`@tturbo` macro turns on multi-threading. Note for this to function, Julia needs to be started with multi-threading.

In [36]:
function jl_sum_tturbo(A)
    s = zero(eltype(A))
    @tturbo for i in eachindex(A)
      s += A[i]
    end
    s
end

bm = @benchmark jl_sum_tturbo($x)
bm

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  42.791 μs … 138.125 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     45.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   46.621 μs ±   3.054 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▄█▂   ▁▂▁     ▂▁▁▁▁                                 ▁
  ▇▇▄▄▃▅▃▇████▇███████▇▆▇███████▇▇▆▄▅▆▆▆▇██▇▇▆▆▇█▇█▆▆▅▆▅▅▅▄▅▅▅ █
  42.8 μs       Histogram: log(frequency) by time      58.2 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [37]:
benchmark_result["Julia tturbo"] = median(bm.times) / 1e6

0.0455

### Summary

In [38]:
sort(collect(benchmark_result), by = x -> x[2])

11-element Vector{Pair{Any, Any}}:
   "Julia tturbo" => 0.0455
    "Julia turbo" => 0.08975
  "Julia builtin" => 0.16075
   "Python numpy" => 0.17375
              "C" => 0.823958
     "Julia loop" => 0.8295
         "R Rcpp" => 0.8523900178261101
      "R builtin" => 1.3731925282627344
         "R loop" => 7.598653028253466
 "Python builtin" => 38.883459
    "Python loop" => 47.732083

* `C`, `R builtin` are the baseline C performance (gold standard).

* `Python builtin` and `Python loop` are >50 fold slower than C because the loop is interpreted.

* `R loop` is about 10 times slower than C and indicates the performance of JIT bytecode generated by its `compiler` package (turned on by default since R v3.4.0 (Apr 2017)). 

* `Julia loop` is close to C performance, because Julia code is JIT compiled.

* `Julia builtin`, `Python numpy`, and `Julia turbo` are 4-5 fold faster than C because of SIMD.

* `@turbo` (SIMD) and `@tturbo` (multithreading+SIMD), offered by the LoopVectorization.jl package. yields the top performance on this machine.

## Take home message for computational scientists

- High-level language (R, Python, Matlab) programmers should be familiar with existing high-performance packages.  Don't reinvent wheels.  
    - R: RcppEigen, tidyverse, ...   
    - Python: numpy, scipy, ...  

- In most research projects, looping is unavoidable. Then we need to use a low-level language.  
    - R: Rcpp, ...  
    - Python: Cython, ...  
    
- In this course, we use Julia, which circumvents the two language problem. So we can spend more time on algorithms, not on juggling $\ge 2$ languages.  